In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import bioframe 
import pandas as pd
import numpy as np 
import os
import json
from io import StringIO
import random

import pysam
import h5py

from Bio import motifs
from Bio import pairwise2
from Bio.Seq import Seq

from matplotlib.colors import ListedColormap
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster, optimal_leaf_ordering

In [2]:
import sys

sys.path.insert(0, "/home1/smaruj/akita_utils/")

# from akita_utils import *
import akita_utils

2022-08-02 16:21:24.448028: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /spack/apps/linux-centos7-x86_64/gcc-8.3.0/python-3.9.2-uvcroioc4witkp6qf7mbebof5ix4wlb6/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/pmix-3.1.3-3sm6emyqaxapunh7rwbjvtaqoqe2e5z3/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/openmpi-4.0.2-ipm3dnvlbtxawpi4ifz7jma6jgr7mexq/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/openblas-0.3.8-2no6mfziiclwxb7lstxoos335gnhjpes/lib:/spack/apps/gcc/8.3.0/lib64::/home1/smaruj/software/GSL/lib:/home1/smaruj/software/HTSLIB/lib
2022-08-02 16:21:24.448067: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
genome_open = pysam.Fastafile("/project/fudenber_735/genomes/mm10/mm10.fa")

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
print("Tensorflow: ", tf.__version__)

from basenji import dataset, seqnn, dna_io, stream

Tensorflow:  2.9.1


In [5]:
# NOTE
# head_i = 0 #human
# head_i = 1 #mouse
#

head_i = 1 #mouse
# head_i = 0
model_num = 1 #which fold to use


#base_dir = '/project/fudenber_735/backup/DNN_HiC/human-mouse_5-16-21/'
#model_dir = base_dir+"/f"+str(model_num)+"_c0/train/"

base_dir = "/project/fudenber_735/tensorflow_models/akita/v2/models/"
model_dir = base_dir + "/f" + str(model_num) + "c0/train/"
model_file  = model_dir + "/model" + str(head_i) + "_best.h5"


# model_dir = '/home1/fudenber/repositories/basenji/manuscripts/akita/'
# model_file = model_dir+'/model_best.h5'

params_file = model_dir + "/params.json"
# params_file -> json (dict) with model's parameters
with open(params_file) as params_open:
    params = json.load(params_open)
    params_model = params["model"]
    params_train = params["train"]
seq_length = params_model["seq_length"]
params_model["verbose"] = False

seqnn_model = seqnn.SeqNN(params_model)
print("built")

seqnn_model.restore(model_file, head_i=head_i)    # model with the mouse head
print("restored")

print(seq_length)

2022-08-02 16:21:31.204020: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /spack/apps/linux-centos7-x86_64/gcc-8.3.0/python-3.9.2-uvcroioc4witkp6qf7mbebof5ix4wlb6/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/pmix-3.1.3-3sm6emyqaxapunh7rwbjvtaqoqe2e5z3/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/openmpi-4.0.2-ipm3dnvlbtxawpi4ifz7jma6jgr7mexq/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/openblas-0.3.8-2no6mfziiclwxb7lstxoos335gnhjpes/lib:/spack/apps/gcc/8.3.0/lib64::/home1/smaruj/software/GSL/lib:/home1/smaruj/software/HTSLIB/lib
2022-08-02 16:21:31.204052: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-02 16:21:31.204093: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (b01-12.hpc.usc.edu): /pro

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence (InputLayer)          [(None, 1310720, 4)  0           []                               
                                ]                                                                 
                                                                                                  
 stochastic_reverse_complement   ((None, 1310720, 4)  0          ['sequence[0][0]']               
 (StochasticReverseComplement)  , ())                                                             
                                                                                                  
 stochastic_shift (StochasticSh  (None, 1310720, 4)  0           ['stochastic_reverse_complement[0
 ift)                                                            ][0]']                     

In [6]:
hic_diags = params_model["diagonal_offset"]
try:
    target_crop = params_model["trunk"][-2]["cropping"]
except:
    target_crop = params_model["target_crop"]

print("hic_diags: ", hic_diags) 
print("target_crop: ", target_crop)
print("seq_length: ", seq_length)

target_length_cropped = int((seq_length//2048 - target_crop*2 - hic_diags) * ((seq_length//2048 - target_crop*2 - hic_diags) +1)/2) 
target_map_size = seq_length//2048  - target_crop*2 
triu_tup = np.triu_indices(target_map_size, 2)    # Return the indices for the upper-triangle of an (n, m) array, here k=2 (diagonal offset)
# target_map_size, target_length_cropped, triu_tup[0].shape

print("target_length_cropped: ", target_length_cropped)
print("target_map_size: ", target_map_size)
print("shape of triu_tup[0]: ", triu_tup[0].shape)

hic_diags:  2
target_crop:  64
seq_length:  1310720
target_length_cropped:  130305
target_map_size:  512
shape of triu_tup[0]:  (130305,)


In [7]:
background_file = base_dir + '../analysis/background_seqs.fa'
background_seqs = []
with open(background_file,'r') as f:
  for line in f.readlines():
    if '>' in line: continue
    background_seqs.append(dna_io.dna_1hot(line.strip())) 

In [8]:
background_seqs = background_seqs[0:1]

In [ ]:
# h5_dirs="/project/fudenber_735/tensorflow_models/akita/v2/analysis/permute_boundaries_motifs_ctcf_mm10_model1/scd.h5"
# score_key="SCD"

# dfs = []
# for h5_file in glob.glob(h5_dirs):
#     dfs.append(akita_utils.h5_to_df(h5_file))
# df = dfs[0].copy()
# df[score_key] = np.mean([df[score_key] for df in dfs], axis=0)
# df["span"] = df["span"].str.decode("utf8")
# df = df[df["strand_2"] != "nan"]

In [ ]:
# df.columns

In [ ]:
# df.strand_2.value_counts()

In [12]:
import glob

def pick_by_SCD_range(h5_dirs, score_key="SCD", hi_threshold=15, lo_threshold=10, number=10):
    
    dfs = []
    for h5_file in glob.glob(h5_dirs):
        dfs.append(akita_utils.h5_to_df(h5_file))
    df = dfs[0].copy()
    df[score_key] = np.mean([df[score_key] for df in dfs], axis=0)
    df["span"] = df["span"].str.decode("utf8")
    df = df[df["strand_2"] != "nan"]
    
    selected = df[(df['SCD'] > lo_threshold) & (df['SCD'] <= hi_threshold)]
    ind = list(selected.index)
    picked_ind = random.choices(ind, k=number)
    
    site_df = selected.loc[picked_ind]
    
    seq_coords_df = (site_df[["chrom", "start", "end", "strand_2", score_key]].copy().rename(
        columns={"start_2": "start",
                 "end_2": "end",
                 "strand_2": "strand",
                 score_key: "genomic_" + score_key}
    ))
    seq_coords_df.reset_index(drop=True, inplace=True)
    seq_coords_df = seq_coords_df.astype({'start':'float','end':'float'})
    # seq_coords_df.reset_index(inplace=True)
    # seq_coords_df = bioframe.expand(seq_coords_df, pad=pad_flank)
    print("df prepared")
    
    return seq_coords_df
    

In [13]:
seq_coords_df = pick_by_SCD_range(h5_dirs="/project/fudenber_735/tensorflow_models/akita/v2/analysis/permute_boundaries_motifs_ctcf_mm10_model1/scd.h5")

29 duplicates removed for  /project/fudenber_735/tensorflow_models/akita/v2/analysis/permute_boundaries_motifs_ctcf_mm10_model1/scd.h5
df prepared


In [14]:
seq_coords_df

,chrom,start,end,strand,genomic_SCD
0,chr3,25550000.0,25560000.0,-,10.679688
1,chr19,25330000.0,25340000.0,-,10.312500
2,chr5,114610000.0,114620000.0,-,12.023438
3,chr5,130860000.0,130870000.0,+,13.468750
4,chr9,64120000.0,64130000.0,-,11.867188
5,chr3,54790000.0,54800000.0,+,11.976562
6,chr1,51630000.0,51640000.0,+,13.695312
7,chr16,42910000.0,42920000.0,+,11.445312
8,chr2,23580000.0,23590000.0,-,11.781250
9,chr1,111640000.0,111650000.0,+,11.632812


In [15]:
seq_coords_df.dtypes

chrom           object
start          float64
end            float64
strand          object
genomic_SCD    float16
dtype: object

In [ ]:
# seq_coords_df.iloc[0]

In [ ]:
# seq_coords_df = (
#         site_df[["chrom", "start_2", "end_2", "strand_2", score_key]]
#         .copy()
#         .rename(
#             columns={
#                 "start_2": "start",
#                 "end_2": "end",
#                 "strand_2": "strand",
#                 score_key: "genomic_" + score_key,
#             }
#         )
#     )
#     seq_coords_df.reset_index(drop=True, inplace=True)
#     seq_coords_df.reset_index(inplace=True)
#     seq_coords_df = bioframe.expand(seq_coords_df, pad=pad_flank)
#     print("df prepared")

In [ ]:
# test with one motif
# inde = 8497
# initial_SCD = test["SCD"].loc[inde]
# chrom = test["chrom"].loc[inde]
# start = test["start"].loc[inde]
# end = test["end"].loc[inde]

In [16]:
num_inserts=2
one_side_radius=100
motif_len=19

In [17]:
multi_insert_length = num_inserts * (motif_len + 2 * one_side_radius)

In [18]:
padding = 0

spacer = one_side_radius - padding
        
offsets = []

for i in range(num_inserts):
    offsets.append( seq_length//2 - multi_insert_length//2 + i * (multi_insert_length//2) + spacer)

In [ ]:
# seq_coords_df = test.loc[8497]

In [ ]:
# seq_coords_df["chrom"]

In [ ]:
# WHY THERE'S NO COLUMN "STRAND"???
# strand_2???

In [ ]:
# genome_open.fetch(this_row.chrom, this_row.start-padding, this_row.end+padding)

In [19]:
orientation = "same"

all_inserts = []

for i in seq_coords_df.index:      #iteraton over sequences
    print(i)

    for bi in range(len(background_seqs)):
        seq_1hot = background_seqs[bi].copy()
        this_row = seq_coords_df.loc[i]

        if orientation == "same":
            seq_1hot_CTCF_pad_left = dna_io.dna_1hot(genome_open.fetch(this_row.chrom, this_row.start-padding, this_row.end+padding).upper())

            if this_row.strand == '-': 
                seq_1hot_CTCF_pad_left = dna_io.hot1_rc(seq_1hot_CTCF_pad_left)

            for offset in offsets:
                seq_1hot[offset : offset + len(seq_1hot_CTCF_pad_left)] = seq_1hot_CTCF_pad_left
            # all_inserts.append(seq_1hot)

        elif orientation == "convergent":
            seq_1hot_CTCF_pad_left = dna_io.dna_1hot(genome_open.fetch(this_row.chrom, this_row.start-padding, this_row.end+padding).upper())

            if this_row.strand == '-': 
                seq_1hot_CTCF_pad_left = dna_io.hot1_rc(seq_1hot_CTCF_pad_left)

            seq_1hot_CTCF_pad_right = dna_io.hot1_rc(seq_1hot_CTCF_pad_left)

            for o in range(len(offsets)):
                if o % 2 == 0:
                    seq_1hot[offsets[o] : offsets[o] + len(seq_1hot_CTCF_pad_left)] = seq_1hot_CTCF_pad_left
                else:
                    seq_1hot[offsets[o] : offsets[o] + len(seq_1hot_CTCF_pad_right)] = seq_1hot_CTCF_pad_right
            # all_inserts.append(seq_1hot)

        elif orientation == "divergent":
            seq_1hot_CTCF_pad_right = dna_io.dna_1hot(genome_open.fetch(this_row.chrom, this_row.start-padding, this_row.end+padding).upper())

            if this_row.strand == '-': 
                seq_1hot_CTCF_pad_right = dna_io.hot1_rc(seq_1hot_CTCF_pad_right)

            seq_1hot_CTCF_pad_left = dna_io.hot1_rc(seq_1hot_CTCF_pad_right)

            for o in range(len(offsets)):
                if o % 2 == 0:
                    seq_1hot[offsets[o] : offsets[o] + len(seq_1hot_CTCF_pad_left)] = seq_1hot_CTCF_pad_left
                else:
                    seq_1hot[offsets[o] : offsets[o] + len(seq_1hot_CTCF_pad_right)] = seq_1hot_CTCF_pad_right
        
        all_inserts.append(seq_1hot)

all_inserts = np.array(all_inserts) 

0
1
2
3
4
5
6
7
8
9


In [20]:
all_inserts.shape

(10, 1310720, 4)

In [21]:
pred = seqnn_model.predict(all_inserts, batch_size=10)

1/1 [==============================] - 13s 13s/step


In [24]:
stat = ["SCD"]
nr_bg = 1
nr_seq = 10
nr_targets = 6

SCD_all_sequences = np.zeros(shape=(nr_seq, nr_bg, nr_targets))

for i in seq_coords_df.index:      #iteraton over sequences
    # print(i)
    
    SCD_all_backgrounds = np.zeros(shape=(nr_bg, nr_targets))
    
    for bi in range(nr_bg):      # iteration over background sequences

        insert_pred = pred[i]

        SCD_all_targets = np.zeros(insert_pred.shape[1])        
        
        for target_ind in range(insert_pred.shape[1]):       # iteration over targets

            # print(target_ind, "SCD: ", np.sqrt( (insert_pred**2).sum(axis=0))[target_ind])

            INS_dict = {}
            mat = akita_utils.from_upper_triu(insert_pred.shape[1], target_map_size, hic_diags)

            for stat_matric in stat:

                if stat_matric == "SCD":

                    SCD = np.sqrt( (insert_pred**2).sum(axis=0))[target_ind]      # sequence = i background = bi, target = target_ind
                    SCD_all_targets[target_ind] = SCD

                    # if verbose == True:
                    print("target_ind: ", target_ind, "SCD: ", np.sqrt( (insert_pred**2).sum(axis=0))[target_ind])
        
        SCD_all_backgrounds[bi] = SCD_all_targets
    
    SCD_all_sequences[i] = SCD_all_backgrounds

target_ind:  0 SCD:  11.39
target_ind:  1 SCD:  16.0
target_ind:  2 SCD:  11.48
target_ind:  3 SCD:  11.31
target_ind:  4 SCD:  11.31
target_ind:  5 SCD:  8.01
target_ind:  0 SCD:  11.31
target_ind:  1 SCD:  11.31
target_ind:  2 SCD:  11.31
target_ind:  3 SCD:  10.125
target_ind:  4 SCD:  11.31
target_ind:  5 SCD:  8.0
target_ind:  0 SCD:  22.62
target_ind:  1 SCD:  23.67
target_ind:  2 SCD:  32.0
target_ind:  3 SCD:  29.17
target_ind:  4 SCD:  30.45
target_ind:  5 SCD:  22.62
target_ind:  0 SCD:  16.0
target_ind:  1 SCD:  18.06
target_ind:  2 SCD:  21.11
target_ind:  3 SCD:  16.0
target_ind:  4 SCD:  16.0
target_ind:  5 SCD:  13.15
target_ind:  0 SCD:  22.62
target_ind:  1 SCD:  32.0
target_ind:  2 SCD:  22.81
target_ind:  3 SCD:  22.62
target_ind:  4 SCD:  22.62
target_ind:  5 SCD:  22.62
target_ind:  0 SCD:  21.75
target_ind:  1 SCD:  22.83
target_ind:  2 SCD:  24.19
target_ind:  3 SCD:  22.62
target_ind:  4 SCD:  22.62
target_ind:  5 SCD:  19.47
target_ind:  0 SCD:  8.31
target_ind

In [29]:
SCD_all_sequences.shape

(10, 1, 6)

In [30]:
np.mean(SCD_all_sequences, axis = 2)

array([[11.58333333],
       [10.5625    ],
       [26.7578125 ],
       [16.72005208],
       [24.21875   ],
       [22.24739583],
       [ 9.17447917],
       [26.5859375 ],
       [ 8.57486979],
       [ 9.60286458]])

In [32]:
SCD_all_sequences[5]

array([[21.75    , 22.828125, 24.1875  , 22.625   , 22.625   , 19.46875 ]])

In [ ]:
# this has to be done for every insertion separatelly

motif_len=19
multi_insert_length = num_inserts * (motif_len + 2 * one_side_radius)

padding = 0

spacer = one_side_radius - padding
        
offsets = []

for i in range(num_inserts):
    offsets.append( seq_length//2 - multi_insert_length//2 + i * (multi_insert_length//2) + spacer


In [36]:
# let's get a lift of sequences of motifs

vanilla_motifs = []

for i in seq_coords_df.index:      #iteraton over sequences
    print(i)
    
    this_row = seq_coords_df.loc[i]

    seq_1hot_CTCF = dna_io.dna_1hot(genome_open.fetch(this_row.chrom, this_row.start, this_row.end).upper())

    if this_row.strand == '-': 
        seq_1hot_CTCF = dna_io.hot1_rc(seq_1hot_CTCF)
    
    vanilla_motifs.append(seq_1hot_CTCF)


0
1
2
3
4
5
6
7
8
9


In [40]:
vanilla_motifs[9]

array([[ True, False, False, False],
       [ True, False, False, False],
       [False, False, False,  True],
       ...,
       [ True, False, False, False],
       [False, False, False,  True],
       [False,  True, False, False]])

In [48]:
nucl_dict = {0: "A", 1: "C", 2: "G", 3: "T"}

In [55]:
best_choices = ["" for f in range(len(seq_coords_df))]

In [63]:
flank = "left"
to_check = 3

stat = ["SCD"]
nr_bg = 1
nr_seq = 1
nr_targets = 6

# for j in range(to_check):
    
#     if flank == "left":
#         print(f"Testing -{j+1}")

j = 0

for i in range(len(seq_coords_df[:1])):      #iteraton over sequences
    # print(i)

    all_inserts = []

    for key in nucl_dict:
        print(nucl_dict[key])

        motif_len = 19 + j + 1
        multi_insert_length = num_inserts * (motif_len + 2 * one_side_radius)

        offsets = []

        for i in range(num_inserts):
            offsets.append(seq_length//2 - multi_insert_length//2 + i * (multi_insert_length//2) + one_side_radius)

        # print(best_choices[i])

        for bi in range(len(background_seqs)):
            seq_1hot = background_seqs[bi].copy()
            this_row = seq_coords_df.loc[i]

            if orientation == "same":
                seq_1hot_CTCF_pad_left = dna_io.dna_1hot(nucl_dict[key] + best_choices[i] + genome_open.fetch(this_row.chrom, this_row.start-padding, this_row.end+padding).upper())

                print(dna_io.dna_1hot(nucl_dict[key] + best_choices[i] + genome_open.fetch(this_row.chrom, this_row.start-padding, this_row.end+padding).upper()))

                if this_row.strand == '-': 
                    seq_1hot_CTCF_pad_left = dna_io.hot1_rc(seq_1hot_CTCF_pad_left)
                for offset in offsets:
                    seq_1hot[offset : offset + len(seq_1hot_CTCF_pad_left)] = seq_1hot_CTCF_pad_left
                # all_inserts.append(seq_1hot)

            elif orientation == "convergent":
                seq_1hot_CTCF_pad_left = dna_io.dna_1hot(nucl_dict[key] + best_choices[i] + genome_open.fetch(this_row.chrom, this_row.start-padding, this_row.end+padding).upper())

                if this_row.strand == '-': 
                    seq_1hot_CTCF_pad_left = dna_io.hot1_rc(seq_1hot_CTCF_pad_left)

                seq_1hot_CTCF_pad_right = dna_io.hot1_rc(seq_1hot_CTCF_pad_left)

                for o in range(len(offsets)):
                    if o % 2 == 0:
                        seq_1hot[offsets[o] : offsets[o] + len(seq_1hot_CTCF_pad_left)] = seq_1hot_CTCF_pad_left
                    else:
                        seq_1hot[offsets[o] : offsets[o] + len(seq_1hot_CTCF_pad_right)] = seq_1hot_CTCF_pad_right
                # all_inserts.append(seq_1hot)

            elif orientation == "divergent":
                seq_1hot_CTCF_pad_right = dna_io.dna_1hot(nucl_dict[key] + best_choices[i] + genome_open.fetch(this_row.chrom, this_row.start-padding, this_row.end+padding).upper())

                if this_row.strand == '-': 
                    seq_1hot_CTCF_pad_right = dna_io.hot1_rc(seq_1hot_CTCF_pad_right)

                seq_1hot_CTCF_pad_left = dna_io.hot1_rc(seq_1hot_CTCF_pad_right)

                for o in range(len(offsets)):
                    if o % 2 == 0:
                        seq_1hot[offsets[o] : offsets[o] + len(seq_1hot_CTCF_pad_left)] = seq_1hot_CTCF_pad_left
                    else:
                        seq_1hot[offsets[o] : offsets[o] + len(seq_1hot_CTCF_pad_right)] = seq_1hot_CTCF_pad_right

            all_inserts.append(seq_1hot)

    all_inserts = np.array(all_inserts)
    print(all_inserts.shape)

    pred = seqnn_model.predict(all_inserts, batch_size=10)
    print(pred.shape)

#     SCD_all_sequences = np.zeros(shape=(nr_seq, nr_bg, nr_targets))

#     for i in seq_coords_df.index:      #iteraton over sequences
#         # print(i)

    SCD_all_nucleotides = np.zeros(shape=(4, nr_targets))

    for ni in range(4):      # iteration over nucleotides tested

        insert_pred = pred[i]

        SCD_all_targets = np.zeros(insert_pred.shape[1])        

        for target_ind in range(insert_pred.shape[1]):       # iteration over targets

            # print(target_ind, "SCD: ", np.sqrt( (insert_pred**2).sum(axis=0))[target_ind])

            INS_dict = {}
            mat = akita_utils.from_upper_triu(insert_pred.shape[1], target_map_size, hic_diags)

            for stat_matric in stat:

                if stat_matric == "SCD":

                    SCD = np.sqrt( (insert_pred**2).sum(axis=0))[target_ind]      # sequence = i background = bi, target = target_ind
                    SCD_all_targets[target_ind] = SCD

                    # if verbose == True:
                    print("target_ind: ", target_ind, "SCD: ", np.sqrt( (insert_pred**2).sum(axis=0))[target_ind])

        # SCD_all_nucleotides[ni] = SCD_all_targets

    # SCD_all_sequences[i] = SCD_all_backgrounds

Testing -1
A

[[ True False False False]
 [False False False  True]
 [False False  True False]
 ...
 [ True False False False]
 [False False  True False]
 [False False  True False]]
C

[[False  True False False]
 [False False False  True]
 [False False  True False]
 ...
 [ True False False False]
 [False False  True False]
 [False False  True False]]
G

[[False False  True False]
 [False False False  True]
 [False False  True False]
 ...
 [ True False False False]
 [False False  True False]
 [False False  True False]]
T

[[False False False  True]
 [False False False  True]
 [False False  True False]
 ...
 [ True False False False]
 [False False  True False]
 [False False  True False]]
(4, 1310720, 4)
1/1 [==============================] - 5s 5s/step
(4, 130305, 6)
target_ind:  0 SCD:  8.12
target_ind:  1 SCD:  8.48
target_ind:  2 SCD:  10.25
target_ind:  3 SCD:  8.0
target_ind:  4 SCD:  8.0
target_ind:  5 SCD:  5.67
target_ind:  0 SCD:  8.12
target_ind:  1 SCD:  8.48
target_ind:  2 SC